In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
from lpne.models import DcsfaNmf

MODEL_FILE = "/hpc/home/mk423/Anxiety/FullDataWork/Models/Final_mt_Model_500_epochs.pt"
DATA_PATH = "/work/mk423/Anxiety/"
PROJECT_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Projections/"
FIGURE_PATH = "/hpc/home/mk423/Anxiety/FullDataWork/Figures/"

model = torch.load(MODEL_FILE,map_location="cpu")
model.device="cpu"

old_feature_list = ["X_power_1_2","X_coh_1_2","X_gc_1_2"]
feature_weights = [10,1,1]

epm_dict_file = DATA_PATH + "EPM_{}_dict_May_17.pkl"

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/hpc/home/mk423/.local/lib/python3.7/site-packages/lpne/pipelines/__init__.py:14: UserWarning: Could not load lpne/pipelines/default_params.yaml!
  warnings.warn("Could not load lpne/pipelines/default_params.yaml!")


In [4]:
for i in ["train"]:
    with open(epm_dict_file.format(i),"rb") as f:
        tempDict = pickle.load(f)
        
        X = np.hstack([tempDict[feature]*weight for feature,weight in zip(old_feature_list,feature_weights)])
        y_roi = (tempDict['y_ROI']%2).astype(bool)
        y_in_task_mask = ~tempDict['y_Homecage'].astype(bool)
        y_mouse = np.array(tempDict['y_mouse'])
        y_time = np.array(tempDict['y_time'])
        nan_mask = np.array((tempDict['y_ROI'] > 0))
        y_expDate = np.array(tempDict['y_expDate'])
        s = model.project(X)
        s_comp = s[:,:3] @ model.classifier[0].weight[0].detach().cpu().numpy()
        
        mouse_roi_comp_net_OA_dict = {}
        mouse_roi_1_net_OA_dict = {}
        mouse_roi_2_net_OA_dict = {}
        mouse_roi_3_net_OA_dict = {}

        mouse_roi_comp_net_CA_dict = {}
        mouse_roi_1_net_CA_dict = {}
        mouse_roi_2_net_CA_dict = {}
        mouse_roi_3_net_CA_dict = {}

        for mouse in np.unique(y_mouse):
            task_mouse_mask = np.logical_and(y_in_task_mask,y_mouse==mouse)
            mask_OA = np.logical_and(task_mouse_mask,y_roi==1)
            mask_CA = np.logical_and(task_mouse_mask,y_roi==0)
            
            comp_score_mean_OA = np.mean(s_comp[mask_OA==1])
            net_1_scores_mean_OA = np.mean(s[mask_OA==1,0])
            net_2_scores_mean_OA = np.mean(s[mask_OA==1,1])
            net_3_scores_mean_OA = np.mean(s[mask_OA==1,2])
            
            mouse_roi_comp_net_OA_dict[mouse] = comp_score_mean_OA
            mouse_roi_1_net_OA_dict[mouse] = net_1_scores_mean_OA
            mouse_roi_2_net_OA_dict[mouse] = net_2_scores_mean_OA
            mouse_roi_3_net_OA_dict[mouse] = net_3_scores_mean_OA

            comp_score_mean_CA = np.mean(s_comp[mask_CA==1])
            net_1_scores_mean_CA = np.mean(s[mask_CA==1,0])
            net_2_scores_mean_CA = np.mean(s[mask_CA==1,1])
            net_3_scores_mean_CA = np.mean(s[mask_CA==1,2])

            mouse_roi_comp_net_CA_dict[mouse] = comp_score_mean_CA
            mouse_roi_1_net_CA_dict[mouse] = net_1_scores_mean_CA
            mouse_roi_2_net_CA_dict[mouse] = net_2_scores_mean_CA
            mouse_roi_3_net_CA_dict[mouse] = net_3_scores_mean_CA
        
        mouse_list = [mouse for mouse in np.unique(y_mouse)]
        dump_dict = {
            "mouse": mouse_list,
            "split":[i for mouse in mouse_list],
            "composite avgOAScore":[mouse_roi_comp_net_OA_dict[mouse] for mouse in mouse_list],
            "net 1 avgOAScore":[mouse_roi_1_net_OA_dict[mouse] for mouse in mouse_list],
            "net 2 avgOAScore":[mouse_roi_2_net_OA_dict[mouse] for mouse in mouse_list],
            "net 3 avgOAScore":[mouse_roi_3_net_OA_dict[mouse] for mouse in mouse_list],

            "composite avgCAScore":[mouse_roi_comp_net_CA_dict[mouse] for mouse in mouse_list],
            "net 1 avgCAScore":[mouse_roi_1_net_CA_dict[mouse] for mouse in mouse_list],
            "net 2 avgCAScore":[mouse_roi_2_net_CA_dict[mouse] for mouse in mouse_list],
            "net 3 avgCAScore":[mouse_roi_3_net_CA_dict[mouse] for mouse in mouse_list],
            
            }
        
        df = pd.DataFrame.from_dict(dump_dict)
        #df.to_csv(PROJECT_PATH + "EPM_{}_OA_CA_mean_scores.csv".format(i))
        print(df.head())

        mouse  split  composite avgOAScore  net 1 avgOAScore  \
0  Mouse04201  train              4.809300          0.118325   
1  Mouse04202  train              4.818913          0.078143   
2  Mouse04205  train              3.183583          0.042897   
3  Mouse04215  train              5.923032          0.032799   
4   Mouse0630  train              4.745236          0.138276   

   net 2 avgOAScore  net 3 avgOAScore  composite avgCAScore  net 1 avgCAScore  \
0          0.215256          0.084666              4.528297          0.105960   
1          0.240228          0.066434              4.938036          0.068153   
2          0.162230          0.080796              3.484384          0.060373   
3          0.334806          0.026722              5.832579          0.024893   
4          0.202133          0.028452              4.855256          0.142657   

   net 2 avgCAScore  net 3 avgCAScore  
0          0.205007          0.099852  
1          0.252451          0.084634  
2       

In [5]:
tempDict

{'X_psd': array([[0.00577849, 0.01332061, 0.01577768, ..., 0.00280759, 0.00778864,
         0.01025723],
        [0.02710032, 0.01810321, 0.00539332, ..., 0.00067895, 0.00202881,
         0.00298027],
        [0.05989832, 0.06313662, 0.03484032, ..., 0.00380414, 0.00512969,
         0.0026963 ],
        ...,
        [0.018172  , 0.01415159, 0.03093626, ..., 0.00249193, 0.00256295,
         0.00179294],
        [0.02005424, 0.02193768, 0.01694107, ..., 0.00150093, 0.00118903,
         0.00119264],
        [0.08068492, 0.0702386 , 0.06126685, ..., 0.00163832, 0.00181637,
         0.00217615]]),
 'X_ds': array([[7.57278933e-04, 9.64690698e-04, 1.23021146e-03, ...,
         2.15845881e-04, 1.90188899e-03, 1.32378750e-03],
        [8.41754547e-04, 7.09852017e-03, 2.06977129e-03, ...,
         3.49986833e-04, 4.02684163e-05, 2.57940288e-03],
        [4.92928065e-02, 4.71513718e-02, 6.49778312e-03, ...,
         1.23983918e-04, 1.25206681e-03, 1.20881305e-03],
        ...,
        [1.04249036